In [8]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
from dotenv import load_dotenv

load_dotenv()  

# dan jurafsky's string2string library
from string2string.similarity import JaroSimilarity


from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from prompting.prompts import DataTemplates

In [68]:
df = pd.read_csv("../abstracts_final.csv")

In [69]:
template = ChatPromptTemplate.from_messages(DataTemplates().medicine_abstract_prompt())

In [72]:
def generate_row(text: str, temperature) -> str:
    llm = ChatOpenAI(temperature=temperature)
    chat = LLMChain(llm=llm, prompt=template)
    output = chat.run({"text": text})
    return output

In [108]:
temperatures = [0.7, 0.9]

In [88]:
import concurrent.futures

def generate_wrapper(row, t):
    text = row["texts"]
    return {"temperature": t, "text": generate_row(text, t)}

generations = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for j in range(10):
        for t in temperatures:
            # Prepare arguments for each row
            args = [(row, t) for index, row in df.iterrows()]b

            # Execute the generate_wrapper function with each row in a separate thread
            results = list(executor.map(lambda params: generate_wrapper(*params), args))
            
            # Add results to generations list
            generations.extend(results)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID d675412aa67423dc58f2d9ee3638cad5 in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fe226e1ef25567f2e19d9578176f234f in your message.).
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can r

In [76]:
# generations = []
# while j < 10:
#     for t in temperatures:
#         for i, row in df.iterrows():
#             text = row["texts"]
#             generations.append({"temperature": t, "text": generate_row(text, t)})
#     j += 1

In [122]:
df2 = pd.DataFrame(generations).reset_index()
df2["index"] = df2["index"].apply(lambda x: x % 16)

In [114]:
id_labels = pd.read_csv("../data/qids.csv")

In [120]:
id_labels.reset_index(inplace=True)

In [125]:
df2 = df2.merge(id_labels[["index", "HITId"]], on = "index")

In [111]:
# df[temperatures] = df2.pivot(index="index", columns="temperature", values="text")

In [128]:
df2.to_csv("../data/generated_outputs.csv", index=False)

In [3]:
gen = pd.read_csv("../data/generated_outputs.csv")

In [ ]:
results = ["ai", "ai", "human", "human", "human", "AI", "AI", "AI", "AI", "human"]




In [7]:
for g in gen["text"].sample(10).values:
    print(g)
    print()

A study conducted in Denmark has provided evidence against the theory that the measles, mumps and rubella (MMR) vaccine causes autism. The study was a retrospective cohort study of all children born in Denmark between 1991 and 1998. The MMR vaccination status was obtained from the Danish National Board of Health, and the autism status was obtained from the Danish Psychiatric Central Register. After adjusting for potential confounders, the study found that there was no association between MMR vaccination and the development of autism. The study provides strong evidence against the hypothesis that the MMR vaccine causes autism.

A new study has found that screening mammography is associated with a reduction in the rate of death from breast cancer, but the screening itself only accounts for about a third of the total reduction. The study looked at women between the ages of 50 and 69 years in Norway who were offered screening mammography every two years from 1996 until 2005. Researchers co

### We need to determine how likely the texts are to be generated by ChatGPT.
We will generate 10 texts for each abstract. We will then compare the similarity between the real human answers and GPT-answers. 

1. Select 3 generations and measure the average similarity to the other ChatGPT answers
2. Select the real texts and see how similar they are to the generations.
3. Use the MTurk data and measure how similar the texts are. 